In [1]:
import keras
import keras.backend as K
from keras.models import Model
import tensorflow as tf
import tensorflow.keras.layers  as KL
import tensorflow.keras.models  as KM
from tensorflow.keras.optimizers import SGD
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization , AveragePooling2D
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() 

In [3]:
x_train = x_train/255.0
x_test = x_test/255.0

In [4]:
#Expand the shape of an array.
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

In [5]:
# #AlexNet
# def alexnet(input_shape, n_classes):
#   input = KL.Input(shape=(28, 28, 1))
  
#   # actually batch normalization didn't exist back then
#   # they used LRN (Local Response Normalization) for regularization
#   x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
#   x = BatchNormalization()(x)
#   x = MaxPool2D(3, strides=2)(x)
  
#   x = Conv2D(256, 5, padding='same', activation='relu')(x)
#   x = BatchNormalization()(x)
#   x = MaxPool2D(3, strides=2)(x)
  
#   x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
#   x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
#   x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
#   x = BatchNormalization()(x)
#   x = KL.GlobalMaxPooling2D()(x)
  
#   x = Flatten()(x)
#   x = Dense(4096, activation='relu')(x)
#   x = Dense(4096, activation='relu')(x)
  
#   output = Dense(n_classes , activation='softmax')(x)
  
#   model = Model(input, output)
#   return model


In [7]:
#VGG16
def vgg(input_shape, n_classes):
  
  input = KL.Input(shape=(28, 28, 1))
  
  x = Conv2D(64, 3, padding='same', activation='relu')(input)
  x = Conv2D(64, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(128, 3, padding='same', activation='relu')(x)
  x = Conv2D(128, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  
  output = Dense(n_classes , activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [8]:
input_shape = KL.Input(shape=(28, 28, 1))
n_classes = 10

K.clear_session()
model = vgg(input_shape, n_classes)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 128)       147584    
                                                             

In [10]:
opt = SGD(learning_rate=0.01, momentum=0.9)    
#model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
acc = model.fit(x_train, y_train, epochs=1, batch_size=32)
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss: {0} - Test Acc: {1}".format(test_loss, test_acc*100))

313/313 [==============================] - 43s 128ms/step - loss: 0.0544 - accuracy: 0.9857
Test Loss: 0.05436287820339203 - Test Acc: 98.5700011253357


In [13]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
y_pred = model.predict(x_test)
#accuracy = accuracy_score(y_test, y_pred)
accuracy = accuracy_score(y_test, np.argmax(y_pred, axis=1))
print (accuracy)
precision = precision_score(y_test, np.argmax(y_pred, axis=1), average='macro')
print (precision)
recall = recall_score(y_test, np.argmax(y_pred, axis=1), average='macro')
print(recall)
f1 = f1_score(y_test, np.argmax(y_pred, axis=1), average='macro')
print(f1)

313/313 [==============================] - 41s 130ms/step
0.9857
0.9858832221488264
0.9854588246347864
0.9855940898714124


In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, np.argmax(y_pred, axis=1))

array([[ 974,    0,    1,    0,    0,    0,    1,    0,    4,    0],
       [   0, 1130,    4,    1,    0,    0,    0,    0,    0,    0],
       [   0,    3, 1029,    0,    0,    0,    0,    0,    0,    0],
       [   1,    1,    2,  993,    0,    4,    0,    3,    6,    0],
       [   0,    0,    0,    0,  978,    0,    1,    0,    0,    3],
       [   2,    0,    0,    5,    0,  865,    5,    0,   15,    0],
       [   8,    5,    0,    0,    3,    0,  941,    0,    1,    0],
       [   0,    9,   15,    0,    0,    1,    0,  998,    0,    5],
       [   0,    1,    2,    1,    0,    0,    0,    1,  969,    0],
       [   1,    0,    1,    0,   11,    4,    0,    4,    8,  980]],
      dtype=int64)